In [1]:
import pandas as pd
from pyopenms import MzMLFile, MSExperiment, OnDiscMSExperiment
import numpy as np
import re
import matplotlib.pyplot as plt

In [2]:
df_phospho_all = pd.read_csv("../../data/diann_results/dia_nn_results_filtered/report_filtered_phospho_all_spectra.tsv", delimiter="\t")
mapping_df_phospho_all = pd.read_csv("../../data/230928_JL_Immonium_ions_Modified_DIA_id_mapping.csv", index_col="renamed_id")

In [3]:
def get_id_number(id_string):
    return int(re.findall("[0-9]+", id_string)[-1])

def get_id_number_from_name(spectrum):
    return re.findall("scan=[0-9]+",spectrum.getMetaValue("spectrum title"))[0][5:]

In [4]:
def get_higher_energy_scan_id(id_string):
    id_number = get_id_number(id_string)
    higher_energy_id_number = id_number + 36
    return f"controllerType=0 controllerNumber=1 scan={higher_energy_id_number}"

In [5]:
def get_df_with_original_and_higher_energy_id_mapped(df, exp, mapping_df):
    df = df.copy()
    ms2_spectra = np.array([spectrum for spectrum in exp.getSpectra() if spectrum.getMSLevel() == 2])
    ms2_spectra_in_df = ms2_spectra[df["MS2.Scan"].to_numpy()]
    df.insert(len(df.columns), "remapped_ids", [s.getNativeID() for s in ms2_spectra_in_df])
    df.insert(len(df.columns), "original_ids", [mapping_df.loc[remapped_id]["original_id"] for remapped_id in df["remapped_ids"]])
    higher_energy_id_number = np.array([get_id_number(get_higher_energy_scan_id(id)) for id in df["original_ids"]])
    df.insert(len(df.columns), "higher_energy_ids", higher_energy_id_number)
    return df

In [6]:
exp_phospho_all = MSExperiment()
MzMLFile().load("../../data/230928_JL_Immonium_ions_Modified_DIA_lower_energy.mzML", exp_phospho_all)

In [7]:
df_phospho_all = get_df_with_original_and_higher_energy_id_mapped(df_phospho_all, exp_phospho_all, mapping_df_phospho_all)

In [8]:
phospho_precursors_all = df_phospho_all[df_phospho_all["Modified.Sequence"].str.contains("UniMod:21")]

In [9]:
len(phospho_precursors_all)

222

In [10]:
def get_detected_ions_for_mod(ions_df, amino_acid, mod_name):
    ions_df = ions_df.copy()
    ions_df.insert(len(ions_df.columns), "id_numbers", [get_id_number(id) for id in ions_df["spectrum_id"]])
    return ions_df[np.logical_and(ions_df["amino_acid"] == amino_acid, ions_df["mod_name"] == mod_name)]

In [72]:
detected_ions_df = pd.read_csv("../../data/result_csvs/230928_JL_Immonium_ions_Modified_DIA.mzML_diagnostic_ions_ppm_tolerance_10_snr_threshold_0.csv")
#detected_ions_df = pd.read_csv("../../data/workflow_test/detected_ions.csv")

phospho_detected = get_detected_ions_for_mod(detected_ions_df, "Tyrosine", "Phospho")
threonine_detected = get_detected_ions_for_mod(detected_ions_df, "Threonine", "Phospho")
serine_detected = get_detected_ions_for_mod(detected_ions_df, "Serine", "Phospho")
detected_ids = phospho_detected["id_numbers"]

In [29]:
len(phospho_detected)

19670

In [60]:
in_detected_all = set(phospho_precursors_all["higher_energy_ids"].to_numpy()).intersection(set(detected_ids))
not_in_detected_all = set(phospho_precursors_all["higher_energy_ids"].to_numpy()).difference(set(detected_ids))

In [61]:
len(in_detected_all)

182

In [21]:
len(phospho_precursors_all[phospho_precursors_all["higher_energy_ids"].isin(in_detected_all)]["Modified.Sequence"])

182

In [22]:
phospho_precursors_all[phospho_precursors_all["higher_energy_ids"].isin(not_in_detected_all)]["Modified.Sequence"]

14                           AAVEINVAVLHSY(UniMod:21)QLAK
15                           AAVEINVAVLHSY(UniMod:21)QLAK
33             AGY(UniMod:21)TEKIVIGMDVAASEFY(UniMod:21)R
37                 AINVQEEKIAALQAFADQLIAAGHY(UniMod:21)AK
60                            APVPTGEVY(UniMod:21)FADSFDR
82                ASPSTAGETPSGVKRLPEY(UniMod:21)PQVDDLLLR
94                    AY(UniMod:21)DATHLVKSYPGSQLDILIDQGK
96         AY(UniMod:21)DATHLVKSY(UniMod:21)PGSQLDILIDQGK
119                       DDIIENAPTTHTEEY(UniMod:21)SGEEK
163                              DTVFALVNY(UniMod:21)IFFK
203     EIAEAY(UniMod:21)LGGKVHSAVITVPAY(UniMod:21)FNDSQR
211                ETAEAFLGHPVTNAVITVPAY(UniMod:21)FNDSQR
216                          EY(UniMod:21)KSDSGKPYYYNSQTK
236                  FLEDVKKLY(UniMod:21)HSEAFTVNFGDTEEAK
242                            FNALFAQGNY(UniMod:21)SEAAK
267        GAY(UniMod:21)QNNEIKHIY(UniMod:21)AISSAALSASYK
444                    GTQCARIVEKYGY(UniMod:21)THLSAGELLR
445         GT

In [52]:
phospho_precursors_all[phospho_precursors_all["higher_energy_ids"].isin(not_in_detected_all)]["original_ids"]

4       controllerType=0 controllerNumber=1 scan=28621
14      controllerType=0 controllerNumber=1 scan=75801
15      controllerType=0 controllerNumber=1 scan=75861
24      controllerType=0 controllerNumber=1 scan=36002
25      controllerType=0 controllerNumber=1 scan=35993
                             ...                      
1121    controllerType=0 controllerNumber=1 scan=26659
1124    controllerType=0 controllerNumber=1 scan=50163
1128    controllerType=0 controllerNumber=1 scan=34347
1129    controllerType=0 controllerNumber=1 scan=34322
1130    controllerType=0 controllerNumber=1 scan=34314
Name: original_ids, Length: 206, dtype: object

In [20]:

od_exp = OnDiscMSExperiment()

od_exp.openFile("../../data/230928_JL_Immonium_ions_Modified_DIA.mzML")


True

In [46]:
od_exp_2 = MSExperiment()
MzMLFile().load("../../data/230928_JL_Immonium_ions_Modified_DIA.mzML", od_exp_2)
#MzMLFile().load("../../data/230928_JL_Immonium_ions_Unmodified_DIA.mzML", od_exp_2)

In [32]:
lower = 216.0420 - 10 * 216.0420 / 1e6
higher = 216.0420 + 10 * 216.0420 / 1e6

In [53]:
phospho_precursors_all[phospho_precursors_all["higher_energy_ids"].isin([94235,100441,79121,71973,62414])].to_csv("detections.csv", index=False)

In [54]:
exp_subset = MSExperiment()
spectra_subset = []
for id in phospho_precursors_all[phospho_precursors_all["higher_energy_ids"].isin([94235,100441,79121,71973,62414])]["original_ids"]:
    print(id)
    spectra_subset.append(od_exp.getSpectrumByNativeId(id))
exp_subset.setSpectra(spectra_subset)
MzMLFile().store("detections.mzML", exp_subset)

controllerType=0 controllerNumber=1 scan=94199
controllerType=0 controllerNumber=1 scan=100405
controllerType=0 controllerNumber=1 scan=71937
controllerType=0 controllerNumber=1 scan=62378


In [34]:
count = 0
for id in not_in_detected_all:
    s = od_exp.getSpectrumByNativeId(f"controllerType=0 controllerNumber=1 scan={id}")
    assert(s.getPrecursors()[0].getMetaValue("collision energy") == 50)
    peaks = s.get_peaks()
    if np.max(peaks[1]) / np.mean(peaks[1]) < 0:
        continue
    peaks_mask = np.logical_and(peaks[0] >= lower, peaks[0] <= higher)
    if peaks_mask.sum() > 0:
        # print(id)
        # print(peaks[0][peaks_mask])
        # print(peaks[1][peaks_mask])
        # print(peaks[1][peaks_mask] > peaks[1].mean())
        # print(peaks[1].mean())
        count += 1
    else:
        #print([peak for peak in peaks[0] if 210 < peak < 220])
        print(id)
        print(phospho_precursors_all[phospho_precursors_all["higher_energy_ids"] == id]["Modified.Sequence"].item())


NameError: name 'od_exp' is not defined

In [68]:
potential_threonine = [(sequence, higher_energy_id) for sequence, higher_energy_id in phospho_precursors_all[phospho_precursors_all["higher_energy_ids"].isin(not_in_detected_all)][["Modified.Sequence", "higher_energy_ids"]].to_numpy() if re.findall("T[A-Z]?Y\(UniMod:21\)", sequence) != [] or re.findall("Y\(UniMod:21\)[A-Z]?T", sequence) != []]

In [70]:
potential_serine = [(sequence, higher_energy_id) for sequence, higher_energy_id in phospho_precursors_all[phospho_precursors_all["higher_energy_ids"].isin(not_in_detected_all)][["Modified.Sequence", "higher_energy_ids"]].to_numpy() if re.findall("S[A-Z]?Y\(UniMod:21\)", sequence) != [] or re.findall("Y\(UniMod:21\)[A-Z]?S", sequence) != []]

In [71]:
len(potential_serine)

17

In [45]:
count

1

In [47]:
count = 0
for s in od_exp_2.getSpectra():
    if s.getMSLevel() == 1 or not s.getPrecursors()[0].getMetaValue("collision energy") == 50:
        continue
    peaks = s.get_peaks()
    if np.max(peaks[1]) / np.mean(peaks[1]) < 0:
        continue
    peaks_mask = np.logical_and(peaks[0] >= lower, peaks[0] <= higher)
    if peaks_mask.sum() > 0:
        if peaks[1][peaks_mask] > peaks[1].mean():
        # print(id)
        # print(peaks[0][peaks_mask])
        # print(peaks[1][peaks_mask])
        # print(peaks[1][peaks_mask] > peaks[1].mean())
        # print(peaks[1].mean())
            count += 1

In [33]:
peaks = od_exp.getSpectrumByNativeId("controllerType=0 controllerNumber=1 scan=75801").get_peaks()

In [34]:
peaks[1][peaks[0] == 216.0419158935547]

array([30077.986], dtype=float32)

In [35]:
peaks[0][np.logical_and(peaks[0] >= lower, peaks[0] <= higher)]

array([216.04191589])

In [36]:
peaks[1].mean()

104376.8

In [37]:
print(list([0]))
print(list(od_exp.getSpectrumByNativeId("controllerType=0 controllerNumber=1 scan=75801").get_peaks()[1]))

[0]
[13956.051, 153663.17, 11988.714, 30193.29, 47023.6, 220910.34, 21687.646, 10849.578, 22577.219, 24277.58, 18190.31, 37338.285, 27046.371, 73239.195, 13823.11, 76426.125, 17569.998, 124864.27, 18190.047, 143937.22, 16798.646, 37375.027, 50842.105, 18111.3, 12168.669, 141033.14, 30077.986, 13566.501, 15900.411, 23534.63, 19395.748, 10999.964, 733296.25, 12256.671, 11196.113, 68828.02, 11318.898, 14724.074, 11789.331, 49578.25, 11746.695, 83017.42, 37373.527, 103112.74, 28575.877, 20888.504, 1011462.06, 13844.922, 295433.7, 139702.14, 984720.25, 33904.445, 13913.63, 87177.71, 18814.484, 19761.547, 13651.386, 12964.594, 16536.2, 31769.746, 21546.125, 23854.713, 50930.16, 23188.717, 18002.244, 13705.101, 17764.95, 15727.253, 13622.103, 53048.35, 74769.44, 16912.293, 1642218.8, 58327.684, 17477.479, 114760.55, 189797.94, 60480.402, 21773.975, 606732.7, 21606.592, 24161.906, 69670.64, 11787.918, 32260.732, 16468.244, 10607.904, 13111.973, 109916.164, 10550.842, 17655.4, 19346.078, 46427.

In [38]:
detected_ions_df[detected_ions_df["id_numbers"].isin(not_in_detected_all)]

,spectrum_id,amino_acid,mod_name,type,theoretical_mz,detected_mz,detected_intensity,id_numbers
120885,controllerType=0 controllerNumber=1 scan=43933,Arginine,unmodified,IM-59,70.0651,70.064629,43496.633,43933
120886,controllerType=0 controllerNumber=1 scan=43933,Histidine,unmodified,IM,110.0713,110.070427,49589.910,43933
120887,controllerType=0 controllerNumber=1 scan=43933,Lysine,unmodified,IM-NH3,84.0808,84.080147,24849.756,43933
120888,controllerType=0 controllerNumber=1 scan=43933,Lysine,unmodified,alpha-amino-epsilon-caprolactam,129.1022,129.101196,28455.049,43933
120889,controllerType=0 controllerNumber=1 scan=43933,Lysine,Acetyl,IM-NH3,126.0913,126.090248,14027.154,43933
...,...,...,...,...,...,...,...,...
429447,controllerType=0 controllerNumber=1 scan=106492,Lysine,Acetyl,IM,143.1179,143.116745,13312.277,106492
429448,controllerType=0 controllerNumber=1 scan=106492,Lysine,Formyl,IM,112.0757,112.074844,18067.162,106492
429449,controllerType=0 controllerNumber=1 scan=106492,Lysine,Malonyl,IM,126.0913,126.090378,102972.620,106492
429450,controllerType=0 controllerNumber=1 scan=106492,Proline,unmodified,IM,70.0651,70.064636,49518.406,106492
